In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

%pip install -U "tensorflow-text==2.13.*"
%pip install "tf-models-official==2.13.*"
%pip install transformers
%pip install sentencepiece
%pip install sacremoses
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
# Helps free memory using collect() method.
import gc 
from official.nlp import optimization

tf.get_logger().setLevel('ERROR')

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import datasets
from datasets import Dataset, DatasetDict

import random
import math

from peft import LoraConfig, get_peft_model, TaskType
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 17.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 MB 2.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 84.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 35.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.6.3
    Uninstalling typing_extensions-4.6.3:
      Successfully uninstalled typing_extensions-4.6.3
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting unin

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data Input


In [5]:
train_data = pd.read_csv("/kaggle/input/emonlp-interiit-techsoc-aiclub/train.csv")

train_df, val_df = train_test_split(train_data, test_size=0.2, random_state=42, shuffle=True)
print(train_df.head(5))

train_ds = Dataset.from_pandas(train_df, preserve_index=False)
val_ds = Dataset.from_pandas(val_df, preserve_index=False)
ds = DatasetDict()

ds['train'] = train_ds
ds['validation'] = val_ds
print(ds)

x_train_str = list(train_data.Text)
y_train_str = train_data.Emotion

                                                 Text  Emotion
31  i find every body beautiful and only want peop...      joy
3   i workout every morning before and feel fabulo...      joy
52  i tried to fill it by befriending people that ...     love
17  i crave as i fall into submission and i did no...  sadness
8    i personalities that can feel pain and suffering  sadness
DatasetDict({
    train: Dataset({
        features: ['Text', 'Emotion'],
        num_rows: 48
    })
    validation: Dataset({
        features: ['Text', 'Emotion'],
        num_rows: 12
    })
})


## Processing labels

In [6]:
labels = []
# For counting the number of unique labels.
for x in y_train_str:
    if x not in labels:
        labels.append(x)
        
labels_dictionary = {}

for i in range(len(labels)):
    labels_dictionary[labels[i]]  = i

# Setting up training labels by encoding them with integers.
y_train = np.ones(60)
for i, y in enumerate(y_train_str):
    y_train[i] = labels_dictionary[y]

# Stores the reverse relationship of labels and their encoding, useful after output.
labels_dictionary_reverse = {v: k for k, v in labels_dictionary.items()}
    
print("No. of labels in dataset : ", len(labels))

No. of labels in dataset :  6


## Loading BERT model and preprocessor.

In [5]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1'

tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

bert_model = hub.KerasLayer(tfhub_handle_encoder)
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [4]:
# Freeing up some memory.
del train_data, y_train_str
gc.collect()

30

## Loading Flan T5 and Tokenizer


In [7]:
model_name='google/flan-t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
def tokenize_function(dataset):
    prompt = ['''Identify the sentiment of the following dialogue from the given labels. Provide a one-word sentiment only.
Labels: {}

Dialogue: {}
Sentiment:
'''.format(labels, text) for text in dataset['Text']]
    dataset['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    dataset['labels'] = tokenizer(dataset["Emotion"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return dataset

tokenized_datasets = ds.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['Text', 'Emotion'])

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

## Data Augmentation
I am augmenting data using translation-detranslation. This leads to slight but considerable changes in the sentence structure without change in overall emotion. Doing this with 2 separate languages allows me to increase the size of my dataset by 3 times.

In [6]:
from transformers import MarianMTModel, MarianTokenizer
# Get the name of the first model
first_model_name1 = 'Helsinki-NLP/opus-mt-en-zh'
first_model_name2 = 'Helsinki-NLP/opus-mt-en-fr'

# Get the tokenizer
first_model_tkn1 = MarianTokenizer.from_pretrained(first_model_name1)
first_model_tkn2 = MarianTokenizer.from_pretrained(first_model_name2)

# Load the pretrained model based on the name
first_model1 = MarianMTModel.from_pretrained(first_model_name1)
first_model2 = MarianMTModel.from_pretrained(first_model_name2)

# Get the name of the second model
second_model_name1 = 'Helsinki-NLP/opus-mt-zh-en'
second_model_name2 = 'Helsinki-NLP/opus-mt-fr-en'

# Get the tokenizer
second_model_tkn1 = MarianTokenizer.from_pretrained(second_model_name1)
second_model_tkn2 = MarianTokenizer.from_pretrained(second_model_name2)


# Load the pretrained model based on the name
second_model1 = MarianMTModel.from_pretrained(second_model_name1)
second_model2 = MarianMTModel.from_pretrained(second_model_name2)


def perform_translation(batch_texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    formated_batch_texts = format_batch_texts(language, batch_texts)
    
    # Generate translation using model
    translated = model.generate(**tokenizer(formated_batch_texts, return_tensors="pt", padding=True))

    # Convert the generated tokens indices back into text
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    
    return translated_texts

def format_batch_texts(language_code, batch_texts):
  
  formated_batch = [">>{}<< {}".format(language_code, text) for text in batch_texts]

  return formated_batch


x_train1 = []
x_train_aug = perform_translation(x_train_str, first_model1, first_model_tkn1)
x_train_aug1 = perform_translation(x_train_aug, second_model1, second_model_tkn1)
x_train1.extend(x_train_aug1)

x_train_aug = perform_translation(x_train_str, first_model2, first_model_tkn2)
x_train_aug1 = perform_translation(x_train_aug, second_model2, second_model_tkn2)
x_train1.extend(x_train_aug1)

x_train = []
for x in x_train1:
    x_train.append("".join(ch for ch in x.lower() if (ch.isalnum() or ch.isspace())))
x_train.extend(x_train_str)


## Making train-test split for BERT with data augmented.

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_train, 3*list(y_train), test_size=0.2, random_state=17)

# Fine-tuning Section

## Tensorflow BERT

In [8]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)
net = outputs['pooled_output']
net = tf.keras.layers.Dropout(0.1)(net)
tf.keras.layers.BatchNormalization(synchronized=True)
output = tf.keras.layers.Dense(6, activation='softmax', kernel_regularizer = tf.keras.regularizers.L1L2(l1=0.02, l2=0.01), name='classifier')(net)
classifier_model = tf.keras.Model(text_input, output)

In [9]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False)
metrics = tf.keras.metrics.SparseCategoricalAccuracy()
callbacks = tf.keras.callbacks.EarlyStopping(monitor="loss", verbose=1, patience=3, restore_best_weights=True)

epochs = 70
steps_per_epoch = len(x_train)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [10]:
classifier_model.compile(optimizer = optimizer, metrics = metrics, loss = loss)
classifier_model.fit(x = np.asarray(x_train),y = np.asarray(y_train), epochs = epochs, validation_split = 0.1, callbacks = callbacks)

Epoch 1/70
5/5 [==============================] - 22s 987ms/step - loss: 6.2281 - sparse_categorical_accuracy: 0.1628 - val_loss: 6.2881 - val_sparse_categorical_accuracy: 0.1333
Epoch 2/70
5/5 [==============================] - 2s 487ms/step - loss: 6.2545 - sparse_categorical_accuracy: 0.2093 - val_loss: 6.2742 - val_sparse_categorical_accuracy: 0.1333
Epoch 3/70
5/5 [==============================] - 3s 558ms/step - loss: 6.2141 - sparse_categorical_accuracy: 0.1938 - val_loss: 6.2536 - val_sparse_categorical_accuracy: 0.1333
Epoch 4/70
5/5 [==============================] - 3s 520ms/step - loss: 6.1444 - sparse_categorical_accuracy: 0.1783 - val_loss: 6.2257 - val_sparse_categorical_accuracy: 0.0667
Epoch 5/70
5/5 [==============================] - 2s 447ms/step - loss: 6.1707 - sparse_categorical_accuracy: 0.1628 - val_loss: 6.1963 - val_sparse_categorical_accuracy: 0.0667
Epoch 6/70
5/5 [==============================] - 3s 529ms/step - loss: 6.1330 - sparse_categorical_accuracy:

In [11]:
loss, accuracy = classifier_model.evaluate(np.asarray(x_test), np.asarray(y_test))

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

2/2 [==============================] - 0s 35ms/step - loss: 4.3145 - sparse_categorical_accuracy: 0.8611
Loss: 4.314503192901611
Accuracy: 0.8611111044883728


## Flan-T5 with LORA (HuggingFace)

In [9]:
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)
peft_model = get_peft_model(original_model, 
                            lora_config)

In [10]:
output_dir = f'/kaggle/working/peft-model-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1    
)
    
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets['train'],
)


In [11]:
peft_trainer.train()

peft_model_path="/kaggle/working/peft-model-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,49.250000


('/kaggle/working/peft-model-checkpoint-local/tokenizer_config.json',
 '/kaggle/working/peft-model-checkpoint-local/special_tokens_map.json',
 '/kaggle/working/peft-model-checkpoint-local/spiece.model',
 '/kaggle/working/peft-model-checkpoint-local/added_tokens.json',
 '/kaggle/working/peft-model-checkpoint-local/tokenizer.json')

## Making Predictions.

In [12]:
test_data = pd.read_csv("/kaggle/input/emonlp-interiit-techsoc-aiclub/test.csv")
test_data.head(5)

x_predict = np.asarray(test_data.Text)
predicted_labels = []


In [13]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

peft_model = PeftModel.from_pretrained(peft_model_base, 
                                       '/kaggle/working/peft-model-checkpoint-local', 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [14]:
for x in x_predict:
    index = 200
    prompt = '''Identify the sentiment of the following dialogue from the given labels. Provide a one-word sentiment only.
    Labels: {}

    Dialogue: {}
    Sentiment:
    '''.format(labels, x)
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
    
    predicted_labels.append(peft_model_text_output)
    
emotion = pd.DataFrame({'Emotion' : predicted_labels})
    

In [ ]:
# # y_predict = classifier_model.predict(x_predict, verbose = 1)

# # Creating DataFrame of predictions.
# # predicted_labels = np.argmax(y_predict, axis=1)
# emotion = pd.DataFrame({'Emotion' : predicted_labels})
# emotion = emotion.replace(labels_dictionary_reverse)

In [15]:
# Creating final csv file.
out = pd.DataFrame({'ID': test_data['ID'], 'Emotion' : emotion['Emotion']})
out.to_csv('submission.csv', index=False)
out.head(5)

,ID,Emotion
0,0,joy
1,1,anger
2,2,joy
3,3,love
4,4,joy
